In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torch torchvision torchaudio \
            tqdm numpy opencv-python \
            transformers matplotlib \
            albumentations Pillow \
            scikit-learn seaborn \
            torchmetrics evaluate \
            gdown

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm.notebook import trange, tqdm
import numpy as np
import cv2
from transformers import TimesformerForVideoClassification, AutoImageProcessor, AutoFeatureExtractor, AutoModelForVideoClassification, TrainingArguments, Trainer
import matplotlib.pyplot as plt
import albumentations as A
import random
import torch.nn as nn
import torch.optim as optim
import torchmetrics
import evaluate
from sklearn.metrics import confusion_matrix
import seaborn as sns
import gdown
import zipfile


### Data Preparation

In [ ]:
# Uploading dataset

# Google Drive file ID
file_id = "1BqMBtsuvb6mTpiZUZ9WKcJA8f1hkI2yX"
url = f"https://drive.google.com/uc?id={file_id}"

# Download file
output = "HMDB_simp.zip"
gdown.download(url, output, quiet=False)

# Unzip the file
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall(".")

print("Download and extraction complete!")

In [ ]:
CATEGORY_INDEX = {
    "brush_hair": 0,
    "cartwheel": 1,
    "catch": 2,
    "chew": 3,
    "climb": 4,
    "climb_stairs": 5,
    "draw_sword": 6,
    "eat": 7,
    "fencing": 8,
    "flic_flac": 9,
    "golf": 10,
    "handstand": 11,
    "kiss": 12,
    "pick": 13,
    "pour": 14,
    "pullup": 15,
    "pushup": 16,
    "ride_bike": 17,
    "shoot_bow": 18,
    "shoot_gun": 19,
    "situp": 20,
    "smile": 21,
    "smoke": 22,
    "throw": 23,
    "wave": 24,
}


In [ ]:
# Transformation: Resize to 224x224 and Convert to Tensor
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

def load_sampled_frames(frame_dir, frame_rate=8):
    """
    Load every [frame_rate]-th frame from a directory and apply transformations.
    """
    frame_files = sorted(os.listdir(frame_dir))  # Ensure frames are in order
    sampled_frames = []

    for i in range(0, len(frame_files), frame_rate):
        frame_path = os.path.join(frame_dir, frame_files[i])
        frame = Image.open(frame_path).convert("RGB")  # Convert to RGB
        frame = transform(frame)  # Apply transformations
        sampled_frames.append(frame)

    return sampled_frames  # List of torch tensors

def create_clips(frames, clip_size=8):
    """
    Given a list of sampled frames, create multiple [clip_size]-frame clips.
    """
    clips = []
    if len(frames) < clip_size:
        return clips  # Not enough frames to create a clip
    for i in range(0, len(frames) - clip_size + 1, clip_size):  # Sliding window
        clip = torch.stack(frames[i:i + clip_size])  # Stack into (clip_size, 3, 224, 224)
        clips.append(clip)

    return clips

In [ ]:
DATASET_PATH = "/content/HMDB_simp"

import random

def split_sources(dataset_path, train_ratio=0.8):
    """
    Splits source folders into train and val sets before processing clips.
    Ensures that all clips from a source video stay in the same set.
    """
    train_sources = {}
    val_sources = {}

    for category in os.listdir(dataset_path):  # Iterate over action categories
        category_path = os.path.join(dataset_path, category)
        if not os.path.isdir(category_path):
            continue

        instances = os.listdir(category_path)  # List all source folders (video IDs)
        random.shuffle(instances)  # Shuffle instances before splitting

        split_idx = int(len(instances) * train_ratio)
        train_sources[category] = instances[:split_idx]  # First 80% for training
        val_sources[category] = instances[split_idx:]  # Last 20% for validation

    return train_sources, val_sources


def process_dataset(dataset_path, sources_dict):
    """
    Processes dataset based on a predefined list of sources.
    """
    dataset = []

    for category, instances in tqdm(sources_dict.items()):
        category_path = os.path.join(dataset_path, category)

        for instance in instances:
            instance_path = os.path.join(category_path, instance)
            if not os.path.isdir(instance_path):
                continue  # Skip non-directory files


            # Load sampled frames
            frames = load_sampled_frames(instance_path)

            # Create 8-frame clips
            clips = create_clips(frames)

            for clip in clips:
                dataset.append((clip, CATEGORY_INDEX[category]))  # Store (clip, label)

    return dataset  # List of (clip, label)

class VideoDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        clip, label = self.dataset[idx]
        return clip, torch.tensor(label, dtype=torch.long)

from torch.utils.data import default_collate

class VideoDataCollator:
    """
    Custom data collator for TimeSFormer.
    Converts (clip, label) tuples into a dictionary format.
    """
    def __call__(self, features):
        clips, labels = zip(*features)  # Unpack (clip, label)
        batch = {
            "pixel_values": torch.stack(clips),  # Stack clips into batch
            "labels": torch.tensor(labels, dtype=torch.long)  # Convert labels to tensor
        }
        return batch


In [ ]:
# Split source folders into train & val
train_sources, val_sources = split_sources(DATASET_PATH)

# Process train and val sets separately
train_dataset = process_dataset(DATASET_PATH, train_sources)
val_dataset = process_dataset(DATASET_PATH, val_sources)

dataset_size = len(train_dataset) + len(val_dataset)

print(f"Total clips: {dataset_size}, Train: {len(train_dataset)}, Val: {len(val_dataset)}")

## Load model

In [ ]:
extractor = AutoFeatureExtractor.from_pretrained("facebook/timesformer-base-finetuned-k400")
model = AutoModelForVideoClassification.from_pretrained(
    "facebook/timesformer-base-finetuned-k400",
    num_labels=len(CATEGORY_INDEX),  # Adjust for our dataset
    ignore_mismatched_sizes=True,
)

# Send model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model on: {device}")

In [ ]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
top5_metric = torchmetrics.classification.Accuracy(top_k=5, task="multiclass", num_classes=len(CATEGORY_INDEX)).to(device)

# Metrics Function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.tensor(logits).argmax(dim=1)

    # Compute Accuracy
    top1_acc = accuracy_metric.compute(predictions=predictions.numpy(), references=labels)["accuracy"]

    # Compute Top-5 Accuracy
    top5_acc = top5_metric(torch.tensor(logits).to(device), torch.tensor(labels).to(device)).item()

    # Compute F1-score (macro)
    f1 = f1_metric.compute(predictions=predictions.numpy(), references=labels, average="macro")["f1"]

    # Confusion Matrix
    INDEX_CATEGORY = {v: k for k, v in CATEGORY_INDEX.items()}
    cm = confusion_matrix(labels, predictions, normalize="true",labels=list(CATEGORY_INDEX.values()))

    # Plot confusion matrix with labeled axes
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=0.5)
    sns.heatmap(cm, annot=True, fmt=".2f", cmap='Blues',
            xticklabels=list(CATEGORY_INDEX.keys()),
            yticklabels=list(CATEGORY_INDEX.keys()))
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")
    plt.show()

    return {
        "accuracy": top1_acc,
        "top-5 accuracy": top5_acc,
        "f1-score": f1
    }


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./timesformer_output",  # Save checkpoints
    eval_strategy="epoch",  # Evaluate after every epoch
    save_strategy="epoch",  # Save model after each epoch
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",  # TensorBoard logs
    logging_steps=10,
    save_total_limit=2,  # Keep only last 2 checkpoints
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False
)

data_collator = VideoDataCollator()

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=extractor,  # Feature extractor
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

# Train Model
trainer.train()

###Experiments with a trained model.

Extract files from the timesformer_output.zip archive to the left bar on the content level. Model.safetensors and optimizer.pt may take a bit longer to be uploaded.

In [ ]:
# Loading our trained model
model = AutoModelForVideoClassification.from_pretrained(
    "/content/",  # Path where model.safetensors and config.json are
    num_labels=25,
    ignore_mismatched_sizes=True
)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# validation dataset loader
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=4)

# just inference
all_preds = []
all_labels = []
with torch.no_grad():
    for videos, labels in val_loader:
        videos = videos.to(device)
        labels = labels.to(device)

        outputs = model(videos)
        preds = torch.argmax(outputs.logits, dim=1)
        print(preds)
        print(labels)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

In [ ]:
# confusion matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues')
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()